In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

#### Introducing data

In [ ]:
# Matrix of travels obtained with gravitational model, neutral calibration coeffs (simulating historical data)
grav_init = np.array[[82, 140, 78